In [11]:
import csv
import os
import re

def courtHistory(listHasil):

	csvFolrder = "./"
	csvName = "courtHistory.csv"

	if os.path.exists(csvName):

		f = open(csvName, 'a', newline='\n')
		print(f)
		print("ada")
		w = csv.writer(f)

	else:

		f = open(csvName, 'w', newline='\n')
		print(f)
		print("tidak ada")
    
		w = csv.writer(f)
		w.writerow(('Nomor Putusan', 'Nama Terdakwa 1', 'Nama Terdakwa 2', 'Nama Terdakwa 3', 'Tuntutan Pidana', 'Tuntutan KUHP', 'Tuntukan Hukuman', 'Putusan Pidana', 'Putusan Hukuman', 'Tanggal Putusan', 'Hakim Ketua', 'Hakim Anggota1', 'Hakim Anggota2', 'Panitera', 'Penuntut Umum'))
    
	# menulis file csv
	for s in listHasil:
		w.writerow(s)

	f.close()
	berhasil = "\nCreate Csv file Berhasil\n" 
	return berhasil





In [38]:
import re

def generateEntity(pathFile,fileName):

    print("a")
    folderData = pathFile
    folderHasil = "./OUTPUT/"
    #namaFile = "12Pid.B2015.PNJktTim.txt"
    namaFile = fileName
    namaFileHasil = "O-"+namaFile
    listHasil =[]

    file_putusan = open(folderData+namaFile, "r", encoding='UTF8')
    baca_baris = file_putusan.readlines()

    terdakwa = 0
    tuntutan_pidana = 0
    tuntutanPidana = 0
    tuntutanHukuman = 0
    tindakPidana = 0
    hukuman = 0
    nomor_putusan = 0
    listTerdakwa=[]

    print("")
    print("====== KEPUTUSAN PENGADILAN =====")
    with open(folderHasil + namaFile, "w", encoding="utf-8") as file:

        for baris in baca_baris:
            panjang_baris = len(baris)
            if panjang_baris > 3:
                baris=baris.lower()

                # mendapatkan entitas NOMOR PUTUSAN
                if baris.find("pid.b") >= 0 and nomor_putusan == 0:
                    nomor_putusan +=1
                    eNomor = re.search( r'\d+/(.)*/\d{4}/(.)*',baris, re.M|re.I)	

                    print("Nomor Putusan : " + eNomor.group())
                    file.writelines("Nomor Putusan : " + eNomor.group() + "\n")
                    listHasil.append(eNomor.group().rstrip("\n")) 
                    
                # mendapatkan entitas NAMA TERDAKWA
                if baris.find("nama terdakwa :") >= 0 or baris.find("nama lengkap :") >= 0:
                    terdakwa += 1
                    eTerdakwa = re.search( r'nama (lengkap|terdakwa) :(.*)',baris, re.M|re.I)                
                    terdakwaKe = str(terdakwa)
                    print("Terdakwa Ke : "+terdakwaKe)
                    print("Nama Terdakwa : "+eTerdakwa.group(2))
                    file.writelines("Terdakwa Ke : "+terdakwaKe + "\n")
                    file.writelines("Nama Terdakwa : "+eTerdakwa.group(2) + "\n" + "\n")
                    listTerdakwa.append(eTerdakwa.group(2).rstrip("\n"))

                if (baris.find("terdakwa ditahan") >= 0 or baris.find("telah ditahan") >= 0 or baris.find("terdakwa berada") >= 0 )and terdakwa >= 1:
                    listHasil.append(listTerdakwa[0])
                    if terdakwa == 1: 
                        listHasil.append("")                
                        listHasil.append("")
                    if terdakwa == 2: 
                        listHasil.append(listTerdakwa[1])                
                        listHasil.append("")
                    if terdakwa == 3: 
                        listHasil.append(listTerdakwa[1])                
                        listHasil.append(listTerdakwa[2])
                    terdakwa = 0
                    
                # mendapatkan entitas TUNTUTAN PIDANA
                if baris.find("menyatakan terdakwa") >= 0 and tuntutan_pidana == 0 and tuntutanPidana==0:
                    print("====== TUNTUTAN =====")
                    file.writelines("TUNTUTAN" + "\n")
                    if baris.find("sebagaimana") >= 0:
                        ePidana = re.search( r'melakukan tindak pidana (.*) sebagaimana',baris, re.M|re.I)                
                    else:
                        ePidana = re.search( r'melakukan tindak pidana (.*) ;',baris, re.M|re.I)
                        
                    if ePidana:
                        print("Tindak Pidana : "+ePidana.group(1))    
                        file.writelines("Tindak Pidana : "+ePidana.group(1) + "\n")        
                        listHasil.append(ePidana.group(1))
                    else:
                        print("Tindak Pidana : ")            
                        listHasil.append("")                

                    eKUHP = re.search( r'pasal (.*) kuhp',baris, re.M|re.I)
                    if eKUHP: 
                        print("Melanggar KUHP : " +eKUHP.group())
                        file.writelines("Melanggar KUHP : " +eKUHP.group() + "\n")
                        listHasil.append(eKUHP.group())
                    else:
                        print("Melanggar KUHP : ")
                        file.writelines("Melanggar KUHP : " + "\n")
                        listHasil.append("")                    
                    tuntutanPidana = 1

                if baris.find("menjatuhkan pidana") >= 0 and tuntutan_pidana == 0 and tuntutanHukuman == 0:
                    #print(baris)
                    etuntutan = re.search( r'selama (.*) dikurangi',baris, re.M|re.I)
                    print("Tuntutan Hukuman : "+etuntutan.group(1))
                    file.writelines("Tuntutan Hukuman : "+etuntutan.group(1) + "\n")
                    listHasil.append(etuntutan.group(1))                
                    tuntutan_pidana = 1

                # mendapatkan entitas PUTUSAN PIDANA
                if baris.find("menyatakan terdakwa") >= 0 and baris.find("terbukti") >= 0 and tuntutan_pidana == 1 and tuntutanPidana == 1:
                    print("====== PUTUSAN =====")
                    print(baris)
                    if baris.find("sebagaimana") >= 0:
                        ePutPidana = re.search( r'melakukan tindak pidana (.*) sebagaimana',baris, re.M|re.I)
                    else:
                        ePutPidana = re.search( r'melakukan tindak pidana (.*) ;',baris, re.M|re.I)
                    print("Putusan Pidana : "+ePutPidana.group(1))        
                    file.writelines("Putusan Pidana : "+ePutPidana.group(1))    
                    listHasil.append(ePutPidana.group(1))
                    tuntutanHukuman = 1

                if baris.find("menjatuhkan pidana") >= 0 and tuntutan_pidana == 1 and tuntutanHukuman == 1:
                    #print(baris)
                    if baris.find("dikurangi") >= 0:
                        ePutusan = re.search( r'selama (.*) dikurangi',baris, re.M|re.I)
                    else:
                        ePutusan = re.search( r'selama (.*) ;',baris, re.M|re.I)    
                    print("Putusan Hukuman : "+ePutusan.group(1))
                    file.writelines("Putusan Hukuman : "+ePutusan.group(1))
                    listHasil.append(ePutusan.group(1))
                    #tuntutan_pidana = 1    
            
                # mendapatkan HAKIM KETUA MAJLIS HAKIM            
                if baris.find("diputuskan dalam") >= 0:
                    print("====== PENUTUP =====")
                    cari_tgl_putusan = baris.find("tanggal")
                    cari_tgl_putusan1 = baris.find("oleh")

                    if baris.find(", se") >= 0:
                        cari_hakim_ketua1 = baris.find(", se")
                    else:
                        cari_hakim_ketua1 = baris.find(". se")

                    if baris.find("oleh kami") >= 0:
                        oleh = 9
                    else:
                        oleh = 5

                    if baris.find("ketua majelis hakim, ") >= 0:  
                        cari_hakim_anggota1 = baris.find("ketua majelis hakim, ")+21
                    elif baris.find("hakim ketua majelis, ") >=0:    
                        cari_hakim_anggota1 = baris.find("hakim ketua majelis, ")+21
                    elif baris.find("hakim ketua, ") >=0:
                        cari_hakim_anggota1 = baris.find("hakim ketua, ")+13

                    cari_hakim_anggota2 = baris.find("masing")            
                    hakim_anggota = baris[cari_hakim_anggota1:cari_hakim_anggota2-1]
                    hakim_anggota1 = hakim_anggota[0:hakim_anggota.find(" dan")]
                    hakim_anggota2 = hakim_anggota[hakim_anggota.find(" dan")+5:len(hakim_anggota)]

                    if baris.find("dibantu oleh") >= 0:
                        cari_panitera1 = baris.find("dibantu oleh")+13
                    else:    
                        cari_panitera1 = baris.find("dibantu")+8

                    if baris.find("sebagai panitera pengganti") >= 0:              
                        cari_panitera2 = baris.find("sebagai panitera pengganti")
                    elif baris.find("panitera pengganti") >= 0:              
                        cari_panitera2 = baris.find("panitera pengganti")
                        
                    if baris.find("dihadiri oleh") >= 0:
                        cari_penuntut_umum1 = baris.find("dihadiri oleh")+13
                    elif baris.find("dihadiri") >= 0:
                        cari_penuntut_umum1 = baris.find("dihadiri")+9    

                    if baris.find("jaksa penuntut umum") >= 0:
                        cari_penuntut_umum2 = baris.find("jaksa penuntut umum")
                    elif baris.find("penuntut umum") >= 0:
                        cari_penuntut_umum2 = baris.find("penuntut umum")    
                    
                    tanggalPutusan = baris[cari_tgl_putusan+8:cari_tgl_putusan1]
                    hakimKetua = baris[cari_tgl_putusan1+oleh:cari_hakim_ketua1]
                    hakimAnggota1 = hakim_anggota1
                    hakimAnggota2 = hakim_anggota2
                    panitera = baris[cari_panitera1:cari_panitera2]
                    penuntutUmum = baris[cari_penuntut_umum1:cari_penuntut_umum2]

                    print("Tanggal Putusan : "+tanggalPutusan)
                    print("Hakim Ketua : "+hakimKetua)
                    print("Hakim Anggota 1 : "+hakimAnggota1)
                    print("Hakim Anggota 2 : "+hakimAnggota2)
                    print("Panitera : "+panitera)
                    print("Penuntut Umum : "+penuntutUmum)
                    
                    file.writelines("\n" + "\n" + "PUTUSAN" + "\n")
                    file.writelines("Tanggal Putusan : " + tanggalPutusan + "\n")
                    file.writelines("Hakim Ketua : " + hakimKetua + "\n")
                    file.writelines("Hakim Anggota 1 : "+hakimAnggota1 + "\n")
                    file.writelines("Hakim Anggota 2 : "+hakimAnggota2 + "\n")
                    file.writelines("Panitera : "+panitera + "\n")
                    file.writelines("Penuntut Umum : "+penuntutUmum + "\n")

                    listHasil.append(tanggalPutusan)
                    listHasil.append(hakimKetua)
                    listHasil.append(hakimAnggota1)
                    listHasil.append(hakimAnggota2)
                    listHasil.append(panitera)
                    listHasil.append(penuntutUmum)
        
            
            
    file_putusan.close()
    print("======================")
    return listHasil


# #start_path = "./INPUT/"
# #namaFile = "48Pid.B2015.PNJKTTIM.txt"

# #hasil = generateEntity(start_path, namaFile)
# #print(hasil)


In [ ]:
import re

state_data = {
    0: "tidak ada",
    1: "terdakwa"
}

In [ ]:
def cari_terdakwa(data):
    terdakwa = 1
    hasil_terdakwa = []
    data_arr = data.lower().split("\n")
    index_terdakwa = []
    
    if re.search("terdakwa " + str(terdakwa), data.lower()):
        while re.search("terdakwa " + str(terdakwa), data.lower()):
            terdakwa += 1
        
        terdakwa = 1
        for baris in data_arr:
            if re.search(r"nama (lengkap|terdakwa) :(.*)", baris):
                print(baris.split(":")[1][0:(len(baris.split(":")[1]) - 1)])
        
        
        
def generateEntity(pathFile,fileName):
    
    folderData = pathFile
    folderHasil = "./OUTPUT/"
    #namaFile = "12Pid.B2015.PNJktTim.txt"
    namaFile = fileName
    namaFileHasil = "O-"+namaFile
    listHasil = []
    
    state = state_data[0]
    
    file_putusan = open(folderData+namaFile, "r", encoding='UTF8')
    baca_baris = file_putusan.read()
    cari_terdakwa(baca_baris)
    
    # for baris in baca_baris:
    
    #     # print(baris.split("\n"))
    #     if baris.split("\n")[0] == "berikut dalam perkara Terdakwa :":
    #         if re.search("perkara terdakwa", baris.split("\n")[0].lower()):
    #             state = state_data[1]
    
    #         if state == state_data[1]:
    #             print("data")            
        
        # print(re.search("perkara terdakwa", baris.lower()))


generateEntity("./INPUT/", "output2.txt")
    

 warsono bin tamurdja
 puji priyanto bin moch iswoyo


In [ ]:
import os
# from courtHistoryCsv import courtHistory

def main():

    start_path = "./INPUT/"
    hasil_path = "./OUTPUT/"
    daftarFile = os.listdir(start_path)
    listHasil = []

    # namaFile = "48Pid.B2015.PNJKTTIM.txt"
    # namaFile = "161Pid.B2015.PNJktTim.txt"
    # hasil = generateEntity(start_path, namaFile)
    # listHasil.append(hasil)

    for tiapFile in daftarFile:
        # print(tiapFile)
        hasil = generateEntity(start_path, tiapFile)
        listHasil.append(hasil)
        print("======= ROW HASIL =======")    
        # print(hasil)
        break


	
    print("======= HASIL AKHIR =======")
 

main()


NameError: name 'generateEntity' is not defined

In [8]:
def preproses(data):
    return data.lower().split("\n")

In [256]:
def exctract_terdakwa(data, finder):
    data = data[data.find(finder):]
    nama_raw = data[data.find("nama lengkap") + len("nama lengkap : "):]
    nama_raw = nama_raw[:nama_raw.find("2")].replace(";", "")
    return nama_raw.strip()

def cari_terdakwa(data):
    data = " ".join(" ".join(preproses(data)).split())
    total_terdakwa = 0
    list_terdakwa = []
    while data.find("terdakwa " + str(total_terdakwa + 1)) != -1:
        total_terdakwa += 1
    if total_terdakwa < 1:
        list_terdakwa.append(exctract_terdakwa(data, "perkara terdakwa"))
    else:
        for i in range(1, total_terdakwa + 1):
            list_terdakwa.append((exctract_terdakwa(data, "terdakwa " + str(i))))
    
    return list_terdakwa
            
list_file = os.listdir("./INPUT/")

for filename in list_file:
    with open("./INPUT/" + filename, "r", encoding="utf-8") as file:
        print(cari_terdakwa(file.read()))
    

['terdakwa']
['xxxxxx']
['slamet als amek bin yarkasi']
['sakur bin bukenel (alm)']
['ahmad saefudin bin muflihun']
['ulitomo als. tomo als. umbel bin supardi']
['fajar usmannudin als kuntet bin sambari', 'ulul taufan als ulul bin slamet taufik']
['tanuri bin h. cahyo (alm)']
['terdakwa']


In [317]:
def cari_tuntutan_tp(data):

    data = " ".join(data.lower().split())

    data = data[data.find("melakukan"):] + " "
    data = data[:data.find("sebagaimana")] + " "
    data = data[:data.find("sesuai")] + " "
    
    print(" ".join(data.split()[3:]))


list_file = os.listdir("./INPUT/")

for filename in list_file:
    data = ""
    
    with open("./INPUT/" + filename, "r", encoding="utf-8") as file:
        data = file.read()

    # print(filename)
    (cari_tuntutan_tp(data))
    print()
    # print()

“ membawa pergi seorang wanita yang belum dewasa, tanpa dikehendaki orang tuanya atau walinya tetapi dengan persetujuannya”

“dengan sengaja melakukan tipu muslihat, serangkaian kebohongan, atau membujuk anak melakukan persetubuhan dengannya atau dengan orang lain secara berlanjut ”

pencurian

“penggelapan”

“mengemudikan kendaraan bermotor yang karena kelalaiannya mengakibatkan kecelakaan lalu lintas yang mengakibatkan orang lain meninggal dunia”

“pencurian dalam keadaan memberatkan “

“dengan terang-terangan dan dengan tenaga bersama menggunakan kekerasan terhadap orang yang mengakibatkan luka-luka”

pencurian dengan pemberatan

“dengan sengaja membujuk anak untuk melakukan persetubuhan dengannya, dilakukan oleh pendidik, tenaga kependidikan, yang dilakukan secara berlanjut ”



In [319]:
def cari_putusan_tp(data):
    data = " ".join(data.split())
    start = data.find("MENGADILI:") if data.find("MENGADILI:") != -1 else data.find("M E N G A D I L I:")
    data = data[start:]
    end = data.find("2.")
    data = data[:end]
    
    data = " ".join(data.lower().split())

    data = data[data.find("melakukan"):] + " "
    data = data[:data.find("sebagaimana")] + " "
    data = data[:data.find("sesuai")] + " "
    
    print(" ".join(data.split()[3:]))

for filename in list_file:
    data = ""
    
    with open("./INPUT/" + filename, "r", encoding="utf-8") as file:
        data = file.read()
        (cari_putusan_tp(data))
    
        
    
    

“membawa pergi seorang wanita yang belum dewasa”
"dengan sengaja, membujuk anak untuk melakukan persetubuhan dengannya "
“pencurian”
“penggelapan”
“karena kelalaiannya mengemudikan kendaraan bermotor mengakibatkan orang lain meninggal dunia”;
“pencurian dalam keadaan memberatkan”
“dengan terang-terangan dan tenaga bersama menggunakan kekerasan terhadap orang yang mengakibatkan luka-luka ”
“pencurian dalam keadaan memberatkan ” s ebagaimana dalam dakwaan tunggal penuntut umum;
“dengan sengaja membujuk anak bersetubuh dengannya dilakukan oleh tenaga kependidikan secara berlanjut”


In [ ]:
def cari_kuhp_tp(data):
    data = " ".join(data.split()).lower()
    start = data.find("setelah mendengar pembacaan tuntutan pidana yang diajukan oleh penuntut umum yang pada pokoknya sebagai berikut:")
    data = data[start:]
    
    pattern = [
        "pasal",
        "sebagaimana diatur dan diancam dalam",
    ]
    
    start = -1
    
    for pat in pattern:
        if data.find(pat) != -1:
            data = data[data.find(pat):] + " "
            data = data[:data.find("2.")] + " "
            data = data[:data.find(";")] + " "
            data = data[:data.find("tentang")] + " "
            data = data.strip()
            break

    
    return data

    
for filename in list_file:
    data = ""
    with open("./INPUT/" + filename, "r", encoding="utf-8") as file:
        data = file.read()
        print(cari_kuhp_tp(data))

pasal 83 jo. 76 f undang-undang nomor 35 tahun 2014
pasal 81 ayat (2) undang-undang nomor 17 tahun 2016
pasal pasal 362 kuhpidana
pasal 372 kitab undang-undang hukum pidana sebagaimana dakwaan alternatif pertama penuntut umum.
pasal 310 ayat (4) undang-undang republik indonesia nomor 22 tahun 2009
pasal 363 ayat (2) kuhp dalam surat dakwaan kami
pasal 170 ayat (2) ke-1 kuhpidana
pasal 363 ayat (1) ke-3 kuhpidana
pasal 81 ayat (2), ayat (3) undang undang nomor 17 tahun 2016 jo. undang undang nomor 35 tahun 2014


In [260]:
def cari_hukuman_tp(data):
    data = " ".join(data.split()).lower()
    start = data.find("setelah mendengar pembacaan tuntutan pidana yang diajukan oleh penuntut umum yang pada pokoknya sebagai berikut:")
    data = data[start:]
    
    start = data.find("selama")
    end = data.find("3.")
    hasil = data[start:end].strip()
  
    if hasil[-1] in ".;":
        hasil = hasil[:-1]
        
    return hasil

for filename in list_file:
    data = ""
    with open("./INPUT/" + filename, "r", encoding="utf-8") as file:
        data = file.read()
        print(cari_hukuman_tp(data))

selama 1 (satu) tahun dan 6 (enam) bulan dikurangi selama terdakwa tersebut berada dalam tahanan dengan perintah agar terdakwa tetap ditahan
selama 10 (sepuluh) tahun dikurangi selama terdakwa tersebut berada dalam tahanan dengan perintah agar terdakwa tetap ditahan
selama 1 (satu) tahun 6 (enam) bulan penjara dikurangi dengan masa penahanan sementara yang telah dijalani oleh terdakwa dengan perintah terdakwa tetap ditahan
selama 2 (dua) tahun dikurangkan selama terdakwa berada dalam tahanan dengan perintah agar terdakwa tetap ditahan
selama 1 (satu) tahun dikurangi selama terdakwa tersebut berada dalam tahanan dengan perintah agar terdakwa tetap ditahan
selama 1 (satu) tahun dan 6 (enam) bulan dikurangi selama terdakwa berada dalam tahanan dengan perintah agar terdakwa tetap ditahan
selama 11 (sebelas) bulan , dikurangi selama para terdakwa tersebut berada dalam tahanan dengan perintah agar para terdakwa tetap ditahan
selama 2 (dua) tahun dikurangi masa penahanan yang telah dijalani d

In [184]:
def cari_hukuman_p(data):
    data = " ".join(data.split())
    start = data.find("MENGADILI:") if data.find("MENGADILI:") != -1 else data.find("M E N G A D I L I:")
    data = data[start:]
    start = data.find("selama")
    end = data.find("3.")
    hasil = data[start:end].strip()

    if hasil[-1] in ".;":
        hasil = hasil[:-1]
        
    return hasil

for filename in list_file:
    data = ""
    with open("./INPUT/" + filename, "r", encoding="utf-8") as file:
        data = file.read()
        print(cari_hukuman_p(data))

selama 9 (sembilan) tahun dan pidana denda sejumlah Rp100.000.000,00 (seratus juta rupiah) dengan ketentuan apabila denda tersebut tidak dibayar maka diganti dengan pidana kurungan selama 3 (tiga) bulan
selama 7 (tujuh) bulan dan 20 (dua puluh) hari
selama 1 (satu) tahun dan 10 (sepuluh) bulan
selama 1 (satu) tahun dan 6 (enam) bulan dan T erdakwa 2 Puji Priyanto Bin Moch Iswoyo pidana penjara selama 2 (dua) tahun dan 3 (tiga) bulan


In [261]:
def cari_hakim(data):
    data = " ".join(data.split())
    start = data.find("MENGADILI:") if data.find("MENGADILI:") != -1 else data.find("M E N G A D I L I:")
    data = data[start:]
    start = data.find("6.")
    data = data[start:]
    data = data.lower()
    start = data.find("oleh kami,")
    
    data_hakim = {
        
    }
    
    if start != -1:
        data_hakim["hakim ketua"] = data[start + len("oleh kami,"):data.find("sebagai hakim ketua")].strip()[:-2]
    else:
        start = data.find("oleh")
        data_hakim["hakim ketua"] = data[start + len("oleh"):data.find("sebagai hakim ketua")].strip()[:-2]
    
    
    
    start = data.find("sebagai hakim ketua")        
    end = data.find("masing-masing sebagai hakim anggota")
    hakim_anggota = data[start + len("sebagai hakim ketua, "):end].split("dan")    
    for i in range(len(hakim_anggota)):
        data_hakim["Hakim Anggota " + str(i + 1)] = hakim_anggota[i].strip()
    
    return data_hakim
    
for filename in list_file:
    data = ""
    with open("./INPUT/" + filename, "r", encoding="utf-8") as file:
        data = file.read()
        print(cari_hakim(data))

{'hakim ketua': 'agus maksum mulyohadi, s.h., m.h', 'Hakim Anggota 1': 'nofan hidayat, s.h., m.h.,', 'Hakim Anggota 2': 'budi setyawan, s.h.'}
{'hakim ketua': 'nofan hidayat, s.h., m.h', 'Hakim Anggota 1': 'muhammad taofik, s.h., m.h., budi setyawan, s.h.'}
{'hakim ketua': 'agus maksum mulyohadi, s.h., m.h', 'Hakim Anggota 1': 'budi setyawan, s.h.,', 'Hakim Anggota 2': 'muhammad dede idham, s.h.'}
{'hakim ketua': 'agus maksum mulyohadi, s.h., m.h', 'Hakim Anggota 1': 'nofan hidayat, s.h., m.h.,', 'Hakim Anggota 2': 'budi setyawan, s.h.'}
{'hakim ketua': 'agus maksum mulyohadi, s.h., m.h', 'Hakim Anggota 1': 'nofan hidayat, s.h., m.h.,', 'Hakim Anggota 2': 'budi setyawan, s.h.'}
{'hakim ketua': 'budi setyawan, s.', 'Hakim Anggota 1': 'nofan hidayat, s.h., m.h.,', 'Hakim Anggota 2': 'muhammad taofik, s.h., m.h.,'}
{'hakim ketua': 'kami , nofan hidayat, s.h., m.h', 'Hakim Anggota 1': 'budi setyawan, s.h., muhammad dede idham, s.h., masing- masing sebagai hakim anggota, putusan tersebut di

In [262]:
def cari_tanggal_putusan(data):
    data = " ".join(data.split())
    start = data.find("MENGADILI:") if data.find("MENGADILI:") != -1 else data.find("M E N G A D I L I:")
    data = data[start:]
    start = data.find("6.")
    data = data[start:]
    data = data.lower()
    data = data[data.find("tanggal") + len("tanggal"):]
    data = data[:data.find(",")].strip()
    return data
    
for filename in list_file:
    data = ""
    with open("./INPUT/" + filename, "r", encoding="utf-8") as file:
        data = file.read()
        print(cari_tanggal_putusan(data))

18 maret 2024
14 maret 2024
13 maret 2024
7 maret 2024
5 maret 2024
…..
02 april 2024 oleh kami
2 april 2024
1 april 2024


In [264]:
def cari_panitera(data):
    data = " ".join(data.split())
    start = data.find("MENGADILI:") if data.find("MENGADILI:") != -1 else data.find("M E N G A D I L I:")
    data = data[start:]
    start = data.find("6.")
    data = data[start:]
    data = data.lower()
    data = data[data.find("dibantu oleh") + len("dibantu oleh "):]
    data = data[:data.find("panitera")].strip()
    return data

for filename in list_file:
    data = ""
    with open("./INPUT/" + filename, "r", encoding="utf-8") as file:
        data = file.read()
        print(cari_panitera(data))

siroju munir, s.h., m.h.,
m. evans firmansyah, s.h.,
siroju munir, s.h., m.h.,
m. evans firmansyah, s.h.,
siroju munir, s.h., m.h.,
subagyo, s.h.,
subagyo, s.h.,
m. evans firmansyah, s.h.
m. evans firmansyah, s.h.,


In [265]:
def cari_penuntut(data):
    data = " ".join(data.split())
    start = data.find("MENGADILI:") if data.find("MENGADILI:") != -1 else data.find("M E N G A D I L I:")
    data = data[start:]
    start = data.find("6.")
    data = data[start:]
    data = data.lower()
    data = data[data.find("dihadiri oleh") + len("dihadiri oleh "):]
    data = data[:data.find("penuntut")].strip()
    
    return data

for filename in list_file:
    data = ""
    with open("./INPUT/" + filename, "r", encoding="utf-8") as file:
        data = file.read()
        print(cari_penuntut(data))

eko hertanto, s.h.,m.h.,
eko hertanto, s.h., m.h.,
ida nurliana, s.h.,
muhammad isa yeihansyah, s.h.,m.h.,
eko hertanto, s.h.,m.h.,
ida nurliana, s.h.
eko hertanto, s.h., m.h.,
broto susilo, s.h. , m.h.,
maziyah, s.h.,


In [266]:
def cari_nomor_putusan(data):
    start_index = data.find("Nomor")
    end_index = data.find("DEMI")
    return data[start_index + len("Nomor"):end_index].strip()

for filename in list_file:
    data = ""
    with open("./INPUT/" + filename, "r", encoding="utf-8") as file:
        data = file.read()
        print(cari_nomor_putusan(data))

XX/Pid.Sus/2024/PN Pkl
XXX/Pid.Sus/2023/PN Pkl
36/Pid.B/2024/PN Pkl
9/Pid.B/2024/PN Pkl
12/Pid.Sus/2024/PN Pkl
25/Pid.B/2024/PN Pkl
42/Pid.B/2024/PN Pkl
22/Pid.B/2024/PN Pkl
14/Pid.Sus/2024/PN Pkl
